# Simple Example 

In [ ]:
from common.ETLBase import PipelineBuilder, PipeConfigBuilder

In [ ]:
config = PipeConfigBuilder()
pipe = PipelineBuilder(config)

In [ ]:
config.setups(a=1,b=9)

In [ ]:
config.setups(z=3)

In [ ]:
config.view()

In [ ]:
config.view(summary=False)

In [ ]:
@pipe._func_
def plus_a_b(a=1,b=2):
    return a+b
#pipe._plus_a_b = plus_a_b

In [ ]:
pipe.setup_connection('c = plus_a_b(a=a,b=b)')
pipe.setup_connection('d = plus_a_b(a=a,b=c)')
pipe.setup_connection('e = plus_a_b(d,d)')
pipe.setup_connection('f = plus_a_b(a=b,b=d)')
pipe.view(summary=False)

In [ ]:
pipe.f.get()

In [ ]:
pipe.view(summary=False, option='all')

# Visualization of the Data Pipeline in an Experiment 

In [1]:
from experiments.ex3.connect_pipeline import pipe

In [ ]:
pipe.view(summary=False)

## checking dependency 

In [ ]:
pipe.view_dependency('split_data',summary=False)

## viewing data content, size, and type 

In [ ]:
pipe.view(summary=False, option = 'all')

## viewing data size and type only 

In [ ]:
pipe.view(summary=False, option = 'light')

In [ ]:
pipe.view_dependency('split_data',summary=False, option='light')

# Obtain values of a temporary result of the pipeline 

In [5]:
pipe.test_dataset.get(verbose=True)   

[LOAD] result of "split_data"
 from memory
[RUN] process of "split_data[0]"
[COMPLETE] split_data[0]
[LOAD] result of "split_data"
 from memory
[RUN] process of "split_data[1]"
[COMPLETE] split_data[1]
[LOAD] result of "extract_feature_cols_and_encode_categoricals"
 from memory
[RUN] process of "extract_feature_cols_and_encode_categoricals[1]"
[COMPLETE] extract_feature_cols_and_encode_categoricals[1]
[LOAD] result of "sample_chids"
 from data/sample/sampled_chids.npy
[RUN] process of "build_chid_to_nid_map"
[COMPLETE] build_chid_to_nid_map
[RUN] process of "process"
[COMPLETE] process
[RUN] process of "process[1]"
[COMPLETE] process[1]
[LOAD] result of "extract_feature_cols_and_encode_categoricals"
 from memory
[RUN] process of "extract_feature_cols_and_encode_categoricals[1]"
[COMPLETE] extract_feature_cols_and_encode_categoricals[1]
[LOAD] result of "sample_chids"
 from data/sample/sampled_chids.npy
[RUN] process of "build_chid_to_nid_map"
[COMPLETE] build_chid_to_nid_map
[RUN] proc

# Example of making modification to the pipeline 

# define new function and connect in the end 

In [ ]:
@pipe._func_
def get_length(tensor_dataset):
    return len(tensor_dataset)

pipe.setup_connection('size = get_length(train_dataset)')

In [ ]:
pipe.view(summary=False)

In [ ]:
pipe.size.get()

# modify a function

In [ ]:
@pipe._rep_func_('build_TensorDataset')
def build_TensorDataset(x_dense, x_sparse, objmean, tscnt, label_0):
    return "Hello New TensorDataset Result!"

In [ ]:
pipe.test_dataset.get()

In [ ]:
pipe.train_dataset.get()